### A CAMBIAR:
Tengo que ver cómo me aseguro de que rho no rd -1 ahora que hemos decidido que solo nos interesa la proporcionalidad positiva. POR AHORA HE PUESTO: d= 1 - max(ro, 0) en vez de 1-ro

Si hay alguna version "final": la funcion line transform no necesita a X como argumento!! no lo cambio ahora xq jode las pruebas q he hecho y no es momento pa arreglarlo to

In [5]:
import numpy as np
from matplotlib import pyplot as plt
# we use the following for plotting figures in jupyter
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# GPy: Gaussian processes library
import GPy


# Various distance definitions 

#A
def npoints_more_than(m1, m2, delta = 0):
    return (np.abs(m1-m2)>delta).sum()
#B
def percentage_more_than(m1, m2, delta = 0):
    return 100*(np.abs(m1-m2)>delta).sum()/len(m1)
#C
def sum_difference(m1, m2, delta = 0):
    return np.abs(m1-m2)[np.where(np.abs(m1-m2) > delta)].sum()
#F delta not needed. Just added to fit the mean_dist method called from below
def max_difference(m1, m2, delta = 0):
    return np.amax(np.abs(m1-m2))
#G delta not needed
def avg_difference(m1,m2, delta = 0):
    return np.abs(m1-m2).sum()/len(m1)
# Error cuadratico
def quad_error(m1,m2, delta=0):
    return np.linalg.norm(m1-m2)
#I error medio relativo (a la imagen)
def avg_rel_difference(m1,m2, delta = 0):
    return (np.abs(m1-m2).sum()/len(m1))/(np.amax([m1,m2])-np.amin([m1,m2]))


# Transformations
from scipy.optimize import curve_fit

#line = lambda x, a, b: a*x + b
def line(x, a, b):
    return a * x + b

#finds best a, b such that gp2 = a*gp1 + b
def line_transform(m1, m2, X):
    popt, pcov = curve_fit(line, m1, m2, bounds = ([0,-np.inf], np.inf)) # a must be positive
    #print(popt)
    Tm1 = line(m1, popt[0], popt[1])
    return Tm1


'''
* gp1, gp2 are the processes to be predicted over the domain X
* epsilon1 \in [0,1] is the weight that will be given in the final distance to the distance between gp1 and gp2 means
  epsilon2 is the weight thatll be given to the correlation between the means
  1 - epsilon1 - epsilon2 will be the weight given to the distance between their variance
* transformation is a function with respect to which we decide if mean1 and mean2 are similar.
i.e., distance(mean1, mean2) = 0 iff mean2 = transformation(m1). (transformation could be calculated as a curve fit)
* var_dist and mean_dist are functions to define the distance between both mean vectors/ variance vectors
* delta is a number used as an argument in mean_dist to decide when the distance between two values is negligible. 
e.g., mean_dist(m1,m2) = sum(|m1-m2| when |m1-m2|> delta)
'''
def distance(gp1, gp2, X, 
             epsilon1 = 0.49, epsilon2 = 0.49, delta = 0.1, 
             transformation = line_transform, 
             mean_dist = avg_difference, 
             var_dist = np.linalg.norm,
             ret_intermediate = False):
    intermediate = ""
    m1, v1 = gp1.predict_noiseless(X, full_cov = True)
    m2, v2 = gp2.predict_noiseless(X, full_cov = True)
    m1= m1.flatten()
    m2 = m2.flatten()
    Tm1 = transformation(m1, m2, X)
    d1 = mean_dist(Tm1, m2, delta)
    intermediate = intermediate + "\tdistance between means, " + str(round(d1,5))
    corrmat = np.corrcoef(m1, m2)    
    ro = (corrmat[1][0]).round(5)    
    intermediate = intermediate + "\n\tcorrelat between means, " + str(ro)
    d2 =  1-max(0,ro) # Negative correlation is "as bad as" zero correlation for us
    d3 = var_dist(v1-v2)    
    intermediate = intermediate + "\n\tdistance between vars, " + str(round(d3,5))
    total_distance = epsilon1*d1 + epsilon2*d2 + (1-epsilon1 - epsilon2)*d3
    if ret_intermediate == True:
        return (total_distance, intermediate)
    else:
        return epsilon1*d1 + epsilon2*d2 + (1-epsilon1 - epsilon2)*d3
    #return epsilon1*d1 + (1-epsilon1)*d2
    

In [2]:
namer = lambda t: t.__name__
vnamer = np.vectorize(namer)

# scale Escala x definido en [a,b] para que tenga dominio [a2, b2]
scale = lambda x, a, b, a2, b2: (x-a)*(b2-a2)/(b-a)+a2
@np.vectorize
def scale_from_unity(x, a, b):
    return scale(x, -1,1,a,b)

# benchmark functions
@np.vectorize
def ackley(a = 20, b = 0.2, c = 2*np.pi, *args):
    s1, s2 = 0, 0
    for xi in args:
        xi = scale_from_unity(xi, -32, 32)
        s1 += xi**2
        s2 += np.cos(c*xi)
    s1 = -a*np.exp(-b*s1)
    s2 = -np.exp(s2)
    return a + np.exp(1) + s1 + s2
ackley.__name__ = 'ackley'

@np.vectorize
def ackley_default(*args):
    return ackley(20, 0.2, 2*np.pi, *args)
ackley_default.__name__ = 'ackley default'

@np.vectorize
def griewank(*args):
    s, p = 0, 1
    i = 1
    for xi in args:
        xi = scale_from_unity(xi, -10, 10)
        s += xi**2/4000
        p *= np.cos(xi/np.sqrt(i))
        i += 1
    return s-p+1
griewank.__name__ = 'griewank'

@np.vectorize
def levy(*args):
    s = 0
    args = scale_from_unity(args, -10, 10)
    w = 1 + (args-1)/4
    for wi in w[:-1]:
        s += (wi-1)**2*(1 + 10*np.sin(1 + np.pi*wi)**2)
    s += np.sin(w[0]*np.pi)**2
    s += (w[-1]-1)**2*(1 + np.sin(2*np.pi*w[-1])**2)
    return s
levy.__name__ = 'levy'

@np.vectorize
def easom(*args):
    p = -1
    e = 0
    for xi in args:
        xi = scale_from_unity(xi, -20, 20)
        p *= np.cos(xi)
        e -= (xi-np.pi)**2
    return p*np.exp(e)
easom.__name__ = 'easom'

@np.vectorize
def michalewicz(m = 10, *args):
    s, i = 0, 1
    for xi in args:
        xi = scale_from_unity(xi, 0, np.pi)
        s += np.sin(xi)*np.power(np.sin(i*xi**2/np.pi), 2*m)
        i += 1
    return -1*s
michalewicz.__name__ = 'michalewicz'

@np.vectorize
def michalewicz_default(*args):
    return michalewicz(10, *args)
michalewicz_default.__name__ = 'michalewicz default'

@np.vectorize
def styb_tang(*args):
    s = 0
    for xi in args:
        xi = scale_from_unity(xi, -5, 5)
        s+= np.power(xi, 4)-16*xi**2+5*xi
    return s/2
styb_tang.__name__ = 'styblinski-tang'

@np.vectorize
def sphere(*args):
    s = 0
    for xi in args:
        xi = scale_from_unity(xi, -5,5)
        s += xi**2
    return s
sphere.__name__= 'sphere'

@np.vectorize
def ellipsoid(*args):
    s = 0
    args = scale_from_unity(args, -5,5)
    for i in range(len(args)):
        for j in range(i+1):
            s+= args[j]**2
    return s
ellipsoid.__name__= 'ellipsoid'

@np.vectorize
def sum_diff_pow(*args):
    s = 0
    #args = scale_from_unity(args, -1,1)
    for i in range(len(args)):
        s += np.power(np.abs(args[i]), i+2)
    return s
sum_diff_pow.__name__= 'sum different pows'

@np.vectorize
def sum_squares(*args):
    s = 0
    #args = scale_from_unity(args, -1,1)
    for i in range(len(args)):
        s += (i+1)*args[i]**2
    return s
sum_squares.__name__= 'sum of squares'

@np.vectorize
def boha(x,y):
    x = scale_from_unity(x,-10,10)
    y = scale_from_unity(y,-10,10)
    return x**2 + 2*y**2 -0.3*np.cos(3*np.pi*x)-0.4*np.cos(4*np.pi*y)+0.7
boha.__name__= 'bohachevsky'

@np.vectorize
def boha_n(*args):
    args = scale_from_unity(args,-10,10)
    s = 0
    i = 1
    for arg in args:
        s += i*arg**2
        if i%2 == 0:
            f = np.cos
        else:
            f = np.sin
        s -= (len(args)+i)*0.1*f((len(args)+i)*np.pi*arg)
        i += 1
    return s
boha_n.__name__= 'bohachevsky'

@np.vectorize
def booth(x,y):
    x = scale_from_unity(x,-10,10)
    y = scale_from_unity(y,-10,10)
    return (x+2*y-7)**2+(2*x+y-5)**2
booth.__name = 'booth'